In [1]:
# Code to prepare the coffee data set. 
# pandas is used to load the CSVs; numpy is
# otherwise used throughout.

import numpy as np
import pandas as pd
import statistics

# MGMTMSA 408 – Operations Analytics
# Homework 2 – LP Duality and Revenue  ManagementOptimization
### Name: Joy Lin                                 
### UID: 806304435



# 1 Learning Coffee Preferences
## Part 1: Modeling preferences through rankings
### a) For the same rankings given in the example, calculate the choice probabilities for the set of products {3, 4}.

\begin{equation*}
P(3|{3, 4}) = λ_2 = 0.3 \\
P(4|{3, 4}) = λ_1 = 0.1 \\
P(0|{3, 4}) = λ_3 = 0.6
\end{equation*}

### b) For the same rankings given in the example, calculate the choice probabilities for the set of products {2, 3, 4}.

\begin{equation*}
P(2|{2, 3, 4}) = λ_3 = 0.6 \\
P(3|{2, 3, 4}) = λ_2 = 0.3 \\
P(4|{2, 3, 4}) = λ_1 = 0.1 \\
P(0|{2, 3, 4}) = 0
\end{equation*}

## Part 2: Choice probabilities as a linear system of equations

### a) For the M = 3 assortments in Part 1 (S1 = {1, 4, 5}, S2 = {3, 4}, S3 = {2, 3, 4}), and for the K = 3 customer types (rankings) given in Part 1, write down the A matrix. Your matrix should have 30 rows and 3 columns.

In [152]:
assortments = np.array([[1, 0, 0, 1, 1, 0, 0, 0, 0, 1],
                        [0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
                        [0, 1, 1, 1, 0, 0, 0, 0, 0, 1]])

## Code to set up `permToA` function

In [153]:
# To make the code easier to read, we will use N to denote the total
# number of options, which is 9 + 1 = 10.
n = 9
N = n + 1 
M = assortments.shape[0]

# NB: the function permToA below returns a single column
# of the A matrix. Note that this "column" is actually
# formated as a 2D array (rows are assortments and range from 0
# to M-1, and columns are options which range from 0 to N-1).
# This is a deliberate design choice, as collapsing it into
# a single 1D vector requires a lot of tedious bookkeeping.

def permToA(perm):
    A_column = np.zeros( (M,N) )
    for i in range(M):
        for j in range(N):
            if (assortments[i,perm[j]] > 0.5):
                A_column[i,perm[j]] = 1.0
                break
    return A_column

In [154]:
perm = [[4, 3, 0, 1, 5, 6, 8, 7, 9, 2],   # Type 1
        [3, 2, 4, 0, 1, 5, 6, 8, 7, 9],   # Type 2
        [2, 5, 0, 1, 4, 3, 6, 8, 7, 9]]   # Type 3

perm = np.array(perm) - 1
perm[perm == -1] = 9
perm = perm.tolist()

K = 3
A = np.zeros((M * N, K))

for k, perm_i in enumerate(perm):
    A[:, k] = permToA(perm_i).flatten()

A

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

### b) Using this matrix, compute v = Aλ, where λ = (0.1, 0.3, 0.6). What do you get for v?

In [155]:
lamb = np.array([0.1, 0.3, 0.6])
v = np.dot(A, lamb)
v

array([0. , 0. , 0. , 0.4, 0.6, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3,
       0.1, 0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0.6, 0.3, 0.1, 0. , 0. ,
       0. , 0. , 0. , 0. ])

## Part 3: Understanding the data

In [156]:
assortments = pd.read_csv("coffee_assortments_v3.csv", header = None)
transaction_counts = pd.read_csv("coffee_transaction_counts_v3.csv", header = None)
assortments = np.asarray( assortments)
transaction_counts = np.asarray( transaction_counts)

### a) Which assortment has the most transactions?

In [135]:
assortments_sum = np.sum(transaction_counts, axis = 1)
assortments_sum_max_index = np.argmax(assortments_sum)

print('Assortment with the most transactions: ', assortments[assortments_sum_max_index])

Assortment with the most transactions:  [0 0 0 0 1 0 1 0 1 1]


### b) Which assortment has the least transactions?

In [136]:
assortments_sum = np.sum(transaction_counts, axis = 1)
assortments_sum_min_index = np.argmin(assortments_sum)

print('Assortment with the most transactions: ', assortments[assortments_sum_min_index])

Assortment with the most transactions:  [0 1 0 0 0 0 1 0 0 1]


### c) How many assortments include the Starbucks brand?

In [3]:
count = 0
for assortment_i in assortments:
    if assortment_i[2] == 1:
        count += 1

print(count, 'assortments include the Starbucks brand.')

18 assortments include the Starbucks brand.


### Convert the matrix in coffee transaction counts v3.csv into an array of choice probabilities, i.e., divide each row by its row sum. If you performed this step correctly, then the choice probability of the no-purchase option for the first assortment should be 0.506887.

In [157]:
row_sums = transaction_counts.sum(axis=1)
choice_prob = transaction_counts / row_sums[:, np.newaxis]
choice_prob[0][9]

0.5068870523415978

### d) For assortment m = 10, what fraction of customers purchased the Starbucks brand?

In [14]:
print(choice_prob[9][2], 'of customers purchased the Starbucks brand.')

0.0 of customers purchased the Starbucks brand.


### e) What fraction of customers offered assortment m = 3 chose to not purchase anything?

In [576]:
print(choice_prob[2][9], 'of customers offered assortment m = 3 chose to not purchase anything.')

0.3221957040572792 of customers offered assortment m = 3 chose to not purchase anything.


## Part 4: Estimating the ranking-based model
### a) Explain why solving problem EST is equivalent to solving the nonlinear problem EST-ℓ1. (Hint: suppose that we have an optimal solution (λ, $ϵ^+$, $ϵ^−$) of problem EST. At optimality, what can you say about what the values of $ϵ^+_{m,i}$ and $ϵ^-_{m,i}$ will be? Is it possible for both $ϵ^+_{m,i}$ and $ϵ^-_{m,i}$ to be non-zero?)

At optimality, the values of $ϵ^+_{m,i}$ and $ϵ^-_{m,i}$ will both be 0. It is not possible for both $ϵ^+_{m,i}$ and $ϵ^-_{m,i}$ to be non-zero due to the optimization process that aims to minimize the discrepancies between predicted choice probabilities and historical choice probabilities, which is also the goal of EST-ℓ1, measuring errors and encouraging sparsity.



### b) Formulate and solve problem EST above for these six rankings. What is the optimal objective? What is the optimal solution (λ, $ϵ^+$, $ϵ^−$)?

In [132]:
n = 9
N = n + 1
M = assortments.shape[0]

perm = [
    [1, 2, 7, 0, 6, 8, 9, 3, 5, 4],
    [8, 9, 5, 7, 3, 6, 4, 1, 2, 0],
    [2, 6, 9, 7, 4, 1, 5, 3, 0, 8],
    [6, 3, 2, 1, 0, 5, 4, 7, 9, 8],
    [5, 2, 4, 7, 8, 1, 6, 0, 3, 9],
    [9, 2, 3, 7, 5, 1, 8, 4, 6, 0]
]

perm = np.array(perm) - 1
perm[perm == -1] = 9
perm = perm.tolist()

K = len(perm)

A = []

for i in range(K):
    A.append(permToA(perm[i]))

In [133]:
from gurobipy import *

model = Model()

lambdas = model.addVars(K, lb=0.0, ub=1.0, name="lambdas") 

epsilon_plus = model.addVars(M, N, lb=0.0, name="epsilon_plus") 
epsilon_minus = model.addVars(M, N, lb=0.0, name="epsilon_minus")

v = choice_prob

# Objective
model.setObjective(quicksum(epsilon_plus[m, i] + epsilon_minus[m, i] for m in range(M) for i in range(N)), GRB.MINIMIZE)


# Constraints
for m in range(M):
    for i in range(N):
        model.addConstr(
            quicksum(A[k] * lambdas[k] for k in range(K))[m, i] - epsilon_plus[m, i] + epsilon_minus[m, i] == choice_prob[m, i]
        )
       
        
model.addConstr(sum(lambdas[k] for k in range(K)) == 1, name="sum_lambda")
    
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 501 rows, 1006 columns and 1306 nonzeros
Model fingerprint: 0x8251c26f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 1e+00]
Presolve removed 310 rows and 620 columns
Presolve time: 0.01s
Presolved: 191 rows, 386 columns, 686 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8133860e+01   5.516037e+01   0.000000e+00      0s
     225    4.1829440e+01   0.000000e+00   0.000000e+00      0s

Solved in 225 iterations and 0.03 seconds (0.00 work units)
Optimal objective  4.182943993e+01


In [134]:
# Output the results
lambda_optimal = np.array([lambdas[k].x for k in range(K)])
e_plus_optimal = np.array([epsilon_plus[(m, i)].x for m in range(M) for i in range(N)])
e_minus_optimal = np.array([epsilon_minus[(m, i)].x for m in range(M) for i in range(N)])

print("Optimized Value:", model.objval)
print("Optimal Lambda:", lambda_optimal)
print("Optimal epsilon plus:", e_plus_optimal)
print("Optimal epsilon minus:", e_minus_optimal)

Optimized Value: 41.82943992668715
Optimal Lambda: [0.24780158 0.12518168 0.057407   0.33337267 0.18879182 0.04744526]
Optimal epsilon plus: [0.00000000e+00 4.75369288e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.15177642e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.68587906e-01 3.22511657e-01
 1.56777558e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.18344072e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.67740214e-01 0.00000000e+00 0.00000000e+00
 1.24894235e-01 0.00000000e+00 0.00000000e+00 8.61713706e-03
 0.00000000e+00 0.00000000e+00 3.62688818e-02 2.50803865e-01
 0.00000000e+00 1.32846508e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.37792721e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.66781250e-01 0.00000000e+00 4.41418174e-03
 1.24565422e-02 0.00000000e+00 9.88421041e-02 1.52006398e-01
 0.00000000e+00 0.00000000e+00 5.40711166e-02 0.00000000e+00
 0.00

## Part 5: A random sampling approach

### a) Set your random seed to 200 and randomly generate 1000 rankings. Solve problem EST with these 1000 rankings. What is your optimal objective value? 
#### Hint: You may find it helpful to use the numpy function np.random.permutation(), where np is the numpy module. When called as np.random.permutation(n+1), this function returns the numbers 0,1,. . . ,n in a random order / permutation. This random permutation can then serve as a ranking of the options. You can then use the permToA function to create the corresponding column of the A matrix.

In [132]:
np.random.seed(200)

n = 9
N = n + 1
M = assortments.shape[0]

perm = [np.random.permutation(n + 1) for _ in range(1000)]

K = len(perm)

A = []

for i in range(1000):
    A.append(permToA(perm[i]))

In [133]:
from gurobipy import *

v = choice_prob

lambdas = model.addVars(K, name="lambda", lb=0, vtype=GRB.CONTINUOUS)

epsilon_plus = model.addVars(M, N, name="epsilon_plus", lb=0, vtype=GRB.CONTINUOUS)
epsilon_minus = model.addVars(M, N, name="epsilon_minus", lb=0, vtype=GRB.CONTINUOUS)

# Objective
objective = sum(epsilon_plus[m, i] + epsilon_minus[m, i] for m in range(M) for i in range(n))
model.setObjective(objective, GRB.MINIMIZE)

# Constraints
for m in range(M):
    for i in range(n):
        model.addConstr(sum(A[k] * lambdas[k] for k in range(K))[m,i] - epsilon_plus[m, i] + epsilon_minus[m, i] == v[m, i])

model.addConstr(sum(lambdas[k] for k in range(K)) == 1, name="sum_lambda")

# Optimize the model
model.update()
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1962 rows, 27324 columns and 83533 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 1e+00]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.098380e+01   0.000000e+00      0s
    2071    5.6651820e-01   0.000000e+00   0.000000e+00      0s

Solved in 2071 iterations and 0.27 seconds (0.26 work units)
Optimal objective  5.665182034e-01


In [134]:
# Output the results
lambda_optimal = np.array([lambdas[k].x for k in range(K)])
e_plus_optimal = np.array([epsilon_plus[(m, i)].x for m in range(M) for i in range(N)])
e_minus_optimal = np.array([epsilon_minus[(m, i)].x for m in range(M) for i in range(N)])

print("Optimized Value:", model.objval)
print("Optimal Lambda:", lambda_optimal)
print("Optimal epsilon plus:", e_plus_optimal)
print("Optimal epsilon minus:", e_minus_optimal)

Optimized Value: 0.566518203403911
Optimal Lambda: [0.00000000e+00 0.00000000e+00 2.36662005e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.87123856e-01 0.00000000e+00
 3.91993626e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.06482896e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 4.13014131e-03 0.00000000e+00
 0.00000000e+00 5.38310648e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.08680884e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 4.01022822e-04 5.20386377e-04
 0.00000000e+00 0.00000000e+00 0.0

## Part 6: A column generation approach
### a) How many possible rankings are there of the n + 1 options?

(n+1)!

### b) Explain the meaning of the $z_{i,j}$ decision variables in problem SEP.

= 1 if option i (including non-purchase) is prefered over option j,

= 0 otherwise.


### c) Explain the meaning of each constraint in problem SEP.

- 13b: It is either option i is preferred over option j or the other way around, but one option cannot be preferred and less preferred than another at the same time.
- 13c: Any option cannot be preferred over itself.
- 13d: The rankings cannot be a close loop but acyclical; If one option is preferred over a second option, and the second option is perferred over a third option, then the third option cannot be preferred over the first option.
- 13e: If option i is not included in $m^{th}$ assortment, it cannot be the most preferred out of $m^{th}$ assortment.
- 13f: If option i is the most preferred out of $m^{th}$ assortment, then option i is preferred over option j; If option i is not preferred over option j, then option i is not most preferred out of $m^{th}$ assortment.
- 13g: There can only be one option to be the most preferred out of $m^{th}$ assortment.
- 13h: One option is either preferred or not than another.
- 13i: One option can be either be the most preferred out of $m^{th}$ assortment or not.



### d) Explain how to determine the ranking σ from the values of the $z_{i,j}$ variables.

If the value of the $z_{i,j}$ variable is 1 for a pair of options i and j, then in the ranking σ, option i will be preferred over option j. As we change the pair compositions, we can write out the full ranking by comparing the preferences of all options over each other.

In [158]:
n = 9
N = n + 1
M = assortments.shape[0]

ranking_list = [
    [1, 2, 7, 0, 6, 8, 9, 3, 5, 4],
    [8, 9, 5, 7, 3, 6, 4, 1, 2, 0],
    [2, 6, 9, 7, 4, 1, 5, 3, 0, 8],
    [6, 3, 2, 1, 0, 5, 4, 7, 9, 8],
    [5, 2, 4, 7, 8, 1, 6, 0, 3, 9],
    [9, 2, 3, 7, 5, 1, 8, 4, 6, 0]
]

ranking_list = np.array(ranking_list) - 1
ranking_list[ranking_list == -1] = 9
ranking_list = ranking_list.tolist()

K = len(ranking_list)

A_list = []

for i in range(K):
    A_list.append(permToA(ranking_list[i]))

## Skeleton code to set up `separation` function and complete constraint generation procedure 

In [159]:
# Below is skeleton code to set up your constraint generation procedure.
# Parts of the uncommented code contain ... 's, which are places 
# where you need to fill in the code. 

# =====================================
# First, set up your dual problem here:

from gurobipy import *

m_dual = Model()
p = m_dual.addVars(M, N, lb = -1, ub = 1) 
q = m_dual.addVar()

v = choice_prob

# Add constraints here:

for k in range(K): 
    m_dual.addConstr(sum(A_list[k][m, i] * p[m, i] for m in range(M) for i in range(N)) + q <= 0)

for m in range(M):
    for i in range(N):
        m_dual.addConstr(-p[m, i] <= 1)
        m_dual.addConstr(p[m, i] <= 1)

# (bounds on p_{m,i}, and constraints corresponding to 
# initial six rankings.)
# You may find it helpful to maintain a list, A_list, which
# contains the 2D arrays generated by permToA. 
# You may also find it helpful to maintain a list, ranking_list,
# which contains the rankings corresponding to A_list. 


# Add objective function here:
m_dual.setObjective(quicksum(v[m, i] * p[m, i] for m in range(M) for i in range(N)) + q, GRB.MAXIMIZE)

m_dual.update()
m_dual.optimize()

# =================================================
# Set up your separation subproblem in Gurobi here: 

m_sub = Model()

A = m_sub.addVars(M,N, vtype = GRB.BINARY)
z = m_sub.addVars(N,N, vtype = GRB.BINARY)

# 13b
    
for i in range(N):
    for j in range(N):
        if i != j:
            m_sub.addConstr(z[i, j] + z[j, i] == 1)

# 13c
for i in range(N):
     m_sub.addConstr(z[i, i] == 0)
        
# 13d
for i in range(N):
    for j in range(N):
        if i != j:
            for k in range(N):
                if j != k and k != i:
                    m_sub.addConstr(z[i, j] + z[j, k] + z[k, i] <= 2)

# 13e
for m in range(M):
    for i in range(N):
        if assortments[m, i] == 0:
            m_sub.addConstr(A[m, i] == 0)
      
    # 13f
    for i in range(N):            
        for j in range(N):
            if (i != j) and (assortments[m, i] == 1) and (assortments[m, j] == 1):
                m_sub.addConstr(A[m, i] <= z[i, j])

# 13g
for m in range(M):
    m_sub.addConstr(sum(A[m, i] for i in range(N)) == 1)

m_sub.update()

# =====================================
# Set up your separation function here:

def separation(p_val, q_val):
    m_sub.setObjective(quicksum(p_val[m, i] * A[m, i] for m in range(M) for i in range(N)) + q_val, GRB.MAXIMIZE)
    
    m_sub.update()
    m_sub.optimize()
    
    A_val = np.asarray( [[A[m,i].x for i in range(N)] for m in range(M)] )
    z_val = np.asarray( [[z[i,j].x for j in range(N)] for i in range(N)] )
    
    # Use z_val to determine the ranking:
    ranking = np.argsort(-np.sum(z_val, axis=1))
    
    # Retrieve objective value
    sub_obj = m_sub.objVal
    
    return sub_obj, A_val, ranking



# =======================================================
# Test your separation function below:

np.random.seed(10)
p_val = np.random.randint(-1,1,size = (M,N))
q_val = 0.0

sub_objval, single_A, single_perm = separation(p_val, q_val)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1006 rows, 501 columns and 1306 nonzeros
Model fingerprint: 0xd3b30737
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1000 rows and 315 columns
Presolve time: 0.01s
Presolved: 6 rows, 186 columns, 301 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.2121405e+01   9.900000e+01   0.000000e+00      0s
      15    4.1829440e+01   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.182943993e+01
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.0

In [160]:
print(sub_objval) # Should display -13.0
print(single_perm) # Should display [8 2 5 0 4 3 1 9 7 6]

-13.0
[8 2 5 4 1 0 3 6 9 7]


In [161]:
# =======================================================
# Finally, put your constraint generation procedure here.

m_dual.Params.OutputFlag = 0
m_dual.update()
m_dual.optimize()

dual_obj = m_dual.objval
CG_iter = 0
print("Iteration ", CG_iter, ": ", dual_obj)

p_val = np.asarray( [ [p[i,j].x for j in range(N)] for i in range(M)] )
q_val = q.x

# Solve separation problem for the first time:
sub_objval, single_A, single_perm = separation(p_val, q_val)

# Create the flag isOptimal to decide whether the solution is optimal
# or not (does it satisfy all the dual constraints or not):
isOptimal = (sub_objval <= 0)

while (not isOptimal):
    
    # Add the column of single_A to A_list:
    A_list.append(single_A)
    
    # Add the ranking of single_ranking to ranking_list:
    ranking_list.append(single_perm)
    
    # Add constraint corresponding to this single_A / single_ranking
    # to dual:
    m_dual.addConstr(sum(A_list[K][m, i] * p[m, i] for m in range(M) for i in range(N)) + q <= 0)
    
    K = K + 1
    
    # Update and re-solve the dual:
    m_dual.update()
    m_dual.optimize()
    
    # Retrieve values of dual variables p and q, and dual objective
    # value: 
    dual_obj = m_dual.objval
    p_val = np.asarray( [ [p[i,j].x for j in range(N)] for i in range(M)] )
    q_val = q.x
    
    # Print information about this iteration:
    CG_iter += 1 
    print()
    print("Iteration ", CG_iter, ": ", dual_obj)
    
    # Solve separation problem, and update the isOptimal flag.
    sub_objval, single_A, single_perm = separation(p_val, q_val)
    isOptimal = (sub_objval <= tol)
    # print("sub_objval = ", sub_objval)
    print("isOptimal = ", isOptimal)

Iteration  0 :  41.82943992668715
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2426 rows, 600 columns and 5732 nonzeros
Model fingerprint: 0x8ec71920
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]

Loaded MIP start from previous solve with objective 2

Presolve removed 870 rows and 295 columns
Presolve time: 0.04s
Presolved: 1556 rows, 305 columns, 3516 nonzeros
Variable types: 0 continuous, 305 integer (305 binary)

Root relaxation: objective 8.400000e+01, 61 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd  

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]

Loaded MIP start from previous solve with objective -0

Presolve removed 870 rows and 295 columns
Presolve time: 0.05s
Presolved: 1556 rows, 305 columns, 3516 nonzeros
Variable types: 0 continuous, 305 integer (305 binary)

Root relaxation: objective 5.200000e+01, 194 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   52.00000    0  129   -0.00000   52.00000      -     -    0s
H    0     0                      26.0000000   52.00000   100%     -    0s
H    0     0                      28.0000000   52.00000  85.7%     -    0s
     0     0   45.00000    0  134   28.00000   45.00000  60.7%     -    0s
H    0     0                      30.0000000   38.83333  29.4%     -    0s
     0     0   38.66667    0  151   30.00000   38.66667  28.9%     -    0s
     0     0   38

H    0     0                      23.0000000   29.66667  29.0%     -    0s
     0     0   29.44444    0  191   23.00000   29.44444  28.0%     -    0s
     0     0   29.44444    0  190   23.00000   29.44444  28.0%     -    0s
     0     0   27.35000    0  222   23.00000   27.35000  18.9%     -    0s
     0     0   26.63158    0  199   23.00000   26.63158  15.8%     -    0s
     0     0   26.36111    0  225   23.00000   26.36111  14.6%     -    0s
     0     0   26.33333    0  203   23.00000   26.33333  14.5%     -    0s
     0     0   26.32292    0  225   23.00000   26.32292  14.4%     -    0s
     0     0   26.32292    0  229   23.00000   26.32292  14.4%     -    0s
     0     0   26.06061    0  222   23.00000   26.06061  13.3%     -    0s
     0     0   26.00000    0  201   23.00000   26.00000  13.0%     -    0s
     0     0   26.00000    0  182   23.00000   26.00000  13.0%     -    0s
     0     0   23.00000    0  199   23.00000   23.00000  0.00%     -    0s

Cutting planes:
  Clique


Loaded MIP start from previous solve with objective -0

Presolve removed 870 rows and 295 columns
Presolve time: 0.04s
Presolved: 1556 rows, 305 columns, 3516 nonzeros
Variable types: 0 continuous, 305 integer (305 binary)

Root relaxation: objective 4.700000e+01, 169 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   47.00000    0  132   -0.00000   47.00000      -     -    0s
H    0     0                      12.0000000   47.00000   292%     -    0s
H    0     0                      14.0000000   47.00000   236%     -    0s
     0     0   37.00000    0  140   14.00000   37.00000   164%     -    0s
H    0     0                      16.0000000   37.00000   131%     -    0s
     0     0   33.00000    0  136   16.00000   33.00000   106%     -    0s
     0     0   33.00000    0  130   16.00000   33.00000   106%     -    0s
     0    


Optimal solution found (tolerance 1.00e-04)
Best objective 1.840000000000e+01, best bound 1.840000000000e+01, gap 0.0000%
isOptimal =  False

Iteration  13 :  5.946833943725553
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2426 rows, 600 columns and 5732 nonzeros
Model fingerprint: 0x009810ac
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]

Loaded MIP start from previous solve with objective -0

Presolve removed 813 rows and 287 columns
Presolve time: 0.03s
Presolved: 1613 rows, 313 columns, 3636 nonzeros
Variable types: 0 continuous, 313 integer (313 binary)

Root relaxation: objective 4.600000e+01, 171 iterations, 0.01 seconds (0.

...

     0     0    8.90451    0  206    0.01169    8.90451      -     -    0s
     0     0    7.25754    0  215    0.01169    7.25754      -     -    0s
     0     0    7.08349    0  201    0.01169    7.08349      -     -    0s
     0     0    6.86937    0  205    0.01169    6.86937      -     -    0s
     0     0    6.83863    0  218    0.01169    6.83863      -     -    0s
     0     0    6.82166    0  232    0.01169    6.82166      -     -    0s
     0     0    6.81442    0  220    0.01169    6.81442      -     -    0s
     0     0    6.81411    0  221    0.01169    6.81411      -     -    0s
H    0     0                       0.0626496    6.67895      -     -    0s
     0     0    6.67895    0  189    0.06265    6.67895      -     -    0s
     0     0    6.63766    0  227    0.06265    6.63766      -     -    0s
     0     0    6.62555    0  235    0.06265    6.62555      -     -    0s
     0     0    6.62492    0  235    0.06265    6.62492      -     -    0s
     0     0    6.58889  

     0     2    6.44424    0  267    0.00626    6.44424      -     -    0s
*   69    41              10       0.0293117    6.12389      -  32.2    0s

Cutting planes:
  Gomory: 1
  Clique: 205
  MIR: 2
  Zero half: 20
  RLT: 26
  BQP: 10

Explored 1146 nodes (23184 simplex iterations) in 1.21 seconds (0.88 work units)
Thread count was 8 (of 8 available processors)

Solution count 4: 0.0293117 0.00626309 0.00259658 -1.77636e-15 
No other solutions better than 0.0293117

Optimal solution found (tolerance 1.00e-04)
Best objective 2.931174876905e-02, best bound 2.931174876906e-02, gap 0.0000%
isOptimal =  False

Iteration  381 :  0.49657771692107333
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2426 rows, 600 columns and 5732 nonzeros
Model fingerprint: 0xe8c0f65c
Variable types: 0 continuous, 600 int

isOptimal =  False

Iteration  383 :  0.4965705891994855
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2426 rows, 600 columns and 5732 nonzeros
Model fingerprint: 0xb5eb9427
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]

Loaded MIP start from previous solve with objective -2.66454e-15

Presolve removed 813 rows and 287 columns
Presolve time: 0.04s
Presolved: 1613 rows, 313 columns, 3636 nonzeros
Variable types: 0 continuous, 313 integer (313 binary)

Root relaxation: objective 2.951516e+01, 122 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  D

     0     0   12.65186    0  143   -0.00000   12.65186      -     -    0s
     0     0   12.20809    0  144   -0.00000   12.20809      -     -    0s
     0     0    9.72254    0  137   -0.00000    9.72254      -     -    0s
     0     0    9.44206    0  198   -0.00000    9.44206      -     -    0s
     0     0    9.40071    0  191   -0.00000    9.40071      -     -    0s
     0     0    9.37872    0  196   -0.00000    9.37872      -     -    0s
     0     0    9.37704    0  199   -0.00000    9.37704      -     -    0s
H    0     0                       0.0019356    8.71923      -     -    0s
     0     0    8.71923    0  212    0.00194    8.71923      -     -    0s
     0     0    8.52921    0  208    0.00194    8.52921      -     -    0s
     0     0    8.52921    0  208    0.00194    8.52921      -     -    0s
     0     0    7.41419    0  182    0.00194    7.41419      -     -    0s
     0     0    7.28643    0  222    0.00194    7.28643      -     -    0s
     0     0    7.26307  

     0     0    8.20385    0  186    0.00154    8.20385      -     -    0s
     0     0    7.61194    0  239    0.00154    7.61194      -     -    0s
     0     0    7.60254    0  248    0.00154    7.60254      -     -    0s
     0     0    7.59258    0  236    0.00154    7.59258      -     -    0s
     0     0    7.59233    0  237    0.00154    7.59233      -     -    0s
     0     0    7.43702    0  248    0.00154    7.43702      -     -    0s
     0     0    7.42274    0  252    0.00154    7.42274      -     -    0s
     0     0    7.42040    0  251    0.00154    7.42040      -     -    0s
     0     0    7.34736    0  243    0.00154    7.34736      -     -    0s
     0     0    7.32379    0  246    0.00154    7.32379      -     -    0s
     0     0    7.28978    0  248    0.00154    7.28978      -     -    0s
     0     0    7.27504    0  250    0.00154    7.27504      -     -    0s
     0     0    7.25930    0  251    0.00154    7.25930      -     -    0s
     0     0    7.24650  

### e) What is the optimal objective value?

In [162]:
dual_obj

0.49656346640598703

### f) How many rankings are generated by your constraint generation procedure?

In [163]:
len(ranking_list)

394

## Part 7: Understanding the column generation solution

## Solve the primal problem EST using the rankings generated by your constraint generation procedure.

In [182]:
A = []

K = len(ranking_list)

for i in range(K):
    A.append(permToA(ranking_list[i]))

N = len(single_perm)
M = assortments.shape[0]

In [183]:
from gurobipy import *

model = Model()

v = choice_prob

lambdas = model.addVars(K, name="lambda", lb=0, vtype=GRB.CONTINUOUS)

epsilon_plus = model.addVars(M, N, name="epsilon_plus", lb=0, vtype=GRB.CONTINUOUS)
epsilon_minus = model.addVars(M, N, name="epsilon_minus", lb=0, vtype=GRB.CONTINUOUS)

# Objective
objective = sum(epsilon_plus[m, i] + epsilon_minus[m, i] for m in range(M) for i in range(N))
model.setObjective(objective, GRB.MINIMIZE)

# Constraints
for m in range(M):
    for i in range(N):
        model.addConstr(sum(A[k] * lambdas[k] for k in range(K))[m,i] - epsilon_plus[m, i] + epsilon_minus[m, i] == v[m, i])

model.addConstr(sum(lambdas[k] for k in range(K)) == 1, name="sum_lambda")

# Optimize the model
model.update()
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 501 rows, 1394 columns and 21094 nonzeros
Model fingerprint: 0x48678db1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-02, 1e+00]
Presolve removed 230 rows and 460 columns
Presolve time: 0.02s
Presolved: 271 rows, 934 columns, 20634 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.329423e+01   0.000000e+00      0s
     779    4.9656347e-01   0.000000e+00   0.000000e+00      0s

Solved in 779 iterations and 0.17 seconds (0.13 work units)
Optimal objective  4.965634664e-01


In [184]:
# Output the results
lambda_optimal = np.array([lambdas[k].x for k in range(K)])
e_plus_optimal = np.array([epsilon_plus[(m, i)].x for m in range(M) for i in range(N)])
e_minus_optimal = np.array([epsilon_minus[(m, i)].x for m in range(M) for i in range(N)])

print("Optimized Value:", model.objval)
print("Optimal Lambda:", lambda_optimal)
print("Optimal epsilon plus:", e_plus_optimal)
print("Optimal epsilon minus:", e_minus_optimal)

Optimized Value: 0.4965634664059859
Optimal Lambda: [5.45639918e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.92090395e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.04734978e-02 1.45879873e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.62855991e-03 0.00000000e+00 2.25096365e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.61366533e-02 3.55227549e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.80551995e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.63125261e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.84017250e-03
 6.50626873e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 7.57799462e-04 8.34576694e-03 0.

### a) How many rankings have a non-zero value of $λ_k$ associated with them?

In [185]:
sum(lambda_optimal != 0)

155

### b) What are the top three rankings in terms of $λ_k$? For these three rankings, which are the top three most preferred options?

In [187]:
for i in range(len(ranking_list)):
    if isinstance(ranking_list[i], np.ndarray):
        ranking_list[i] = ranking_list[i].tolist()
        
sorted_indices = np.argsort(lambda_optimal)[::-1]
top_3_indices = sorted_indices[:3]
top_3_rankings = [ranking_list[i] for i in top_3_indices]
print("Top three rankings:", top_3_rankings)

Top three rankings: [[9, 8, 7, 6, 5, 4, 3, 2, 1, 0], [1, 9, 3, 0, 2, 8, 5, 7, 4, 6], [4, 9, 2, 6, 1, 8, 7, 3, 5, 0]]


In [188]:
print('Top three most preferred options for each ranking:')

for ranking in top_3_rankings:
    print(ranking[:3])

Top three most preferred options for each ranking:
[9, 8, 7]
[1, 9, 3]
[4, 9, 2]


### c) What is the average rank of each of the ten options (the n = 9 products plus the no-purchase option), where the average is taken with respect to the rankings and the probability distribution λ in your constraint generation solution? (Lower rank values correspond to more preferred options.)

In [189]:
ave_ranks_per_option = []

for i in range(N):
    ave_ranks_of_option_i = 0
    
    for k in range(K):
        ranks_of_option_i_ranking_k = ranking_list[k].index(i) * lambda_optimal[k]
        ave_ranks_of_option_i = ave_ranks_of_option_i + ranks_of_option_i_ranking_k
    
    ave_ranks_per_option.append(ave_ranks_of_option_i)

data = {'Option': range(N),
        'Average rank': ave_ranks_per_option}
df = pd.DataFrame(data)
print(df)

   Option  Average rank
0       0      4.999857
1       1      3.735251
2       2      4.520811
3       3      5.018357
4       4      4.914700
5       5      5.419680
6       6      4.885636
7       7      4.628118
8       8      4.645332
9       9      2.232258


### d) Suppose that we offer the assortment S = {2, 4, 5, 6}. This corresponds to offering Folgers, Philz, Blue Bottle and Intelligentsia. What are the predicted choice probabilities for the products in this assortment? What is the predicted probability of customers not purchasing anything?

In [190]:
assortment = np.array([[0, 1, 0, 1, 1, 1, 0, 0, 0, 1]])

n = 9
N = n + 1 
M = assortment.shape[0]

perm = ranking_list

K = len(perm)
A_7d = np.zeros((M * N, K))

for k, perm_i in enumerate(perm):
    A_7d[:, k] = permToA(perm_i).flatten()

A_7d

v = np.dot(A_7d, lambda_optimal)
v

array([0.        , 0.39696988, 0.        , 0.        , 0.        ,
       0.        , 0.09614307, 0.        , 0.        , 0.50688705])

In [191]:
print('The predicted choice probabilities for the products in this assortment is')
print(v[:9])
print('The predicted probability of customers not purchasing anything is', v[9])

The predicted choice probabilities for the products in this assortment is
[0.         0.39696988 0.         0.         0.         0.
 0.09614307 0.         0.        ]
The predicted probability of customers not purchasing anything is 0.5068870523415978


# 2 Designing a Sushi Menu

### a) What are the five most preferred sushis for customer 1? Hint: use the command argsort from numpy on a particular row of the utility matrix.

In [322]:
utilities = pd.read_csv("sushi_utilities_mat.csv", header = None)
info = pd.read_csv('sushi_info.csv', header = None)
desc = pd.read_csv('sushi_descriptions.csv', header = None)

utilities = np.asarray( utilities)
info = np.asarray( info)
desc = np.asarray( desc)

In [323]:
sorted_utilities = np.argsort(utilities[0, :])
top_five_indices = sorted_utilities[-5:]
pd.DataFrame(desc[top_five_indices, :]).rename(columns={0: 'Number', 1: 'Name'})

,Number,Name
0,44,inari (fried tofu wrapper)
1,16,samon (salmon)
2,3,maguro (tuna)
3,4,ika (squid)
4,38,negi_toro (fatty flesh of tuna minced to a pas...


### b) What are the five least preferred sushis for customer 2?

In [324]:
sorted_utilities = np.argsort(utilities[1, :])
top_five_indices = sorted_utilities[0:5]
pd.DataFrame(desc[top_five_indices, :]).rename(columns={0: 'Number', 1: 'Name'})

,Number,Name
0,94,karasumi (dried mullet roe)
1,50,komochi_konbu (herring roe & sea tangle)
2,93,kyabia (caviar)
3,15,awabi (abalone)
4,17,kazunoko (herring roe)


### c) For each customer, compute the rank of each of the 100 sushis according to their utilities. Which sushis are the top five, i.e., the five with the best average rank over the 500 customers? What do you notice about the sushi (specifically, the type of fish)?

In [325]:
import numpy as np
temp = np.array([4.8, 2.3, 5.3, 3.9, 1.4, 5.1])
# temp contains the (hypothetical) utilities of 6 products
# Apply argsort once:
temp2 = np.argsort(temp) # [4, 1, 3, 0, 5, 2], products ranked based on increasing utility
temp3 = np.argsort(temp2) # [3, 1, 5, 2, 0, 4] rankings of utilities based on increasing product numbers

In [326]:
nProducts = utilities.shape[1]
nCustomers = utilities.shape[0]
utilities3 = []

for i in range(nCustomers):
    utilities2_i = np.argsort(utilities[i, :])
    utilities3_i = np.argsort(utilities2_i)
    utilities3.append(utilities3_i)

ranking = 0
ranking_ave = np.zeros(nProducts)
for j in range(nProducts):
    for i in range(nCustomers):
        ranking = utilities3[i][j] + ranking
    ranking_ave[j] = ranking
    ranking = 0
ranking_ave = ranking_ave / nCustomers

sorted_utilities = np.argsort(ranking_ave)
top_five_indices = sorted_utilities[-5:]
pd.DataFrame(desc[top_five_indices, :]).rename(columns={0: 'Number', 1: 'Name'})

,Number,Name
0,1,ebi (shrimp)
1,62,negi_toro_maki (roll style of no.37)
2,80,kurumaebi (prawn)
3,3,maguro (tuna)
4,38,negi_toro (fatty flesh of tuna minced to a pas...


The type of fish is either shrimp/prawn or tuna.

### d) Which sushi has the worst average rank over the 500 customers?

In [327]:
ranking = 0
ranking_ave = np.zeros(nProducts)
for j in range(nProducts):
    for i in range(nCustomers):
        ranking = utilities3[i][j] + ranking
    ranking_ave[j] = ranking
    ranking = 0
ranking_ave = ranking_ave / nCustomers

sorted_utilities = np.argsort(ranking_ave)
top_five_indices = sorted_utilities[:1]
pd.DataFrame(desc[top_five_indices, :]).rename(columns={0: 'Number', 1: 'Name'})

,Number,Name
0,93,kyabia (caviar)


### e) Which sushi is the most controversial, as measured by the standard deviation of its rank over the 500 customers?

In [328]:
ranking_sd = []

for i in range(nProducts):
    ranking_sd_i = statistics.stdev([cust[i] for cust in utilities3])
    ranking_sd.append(ranking_sd_i)

sorted_utilities = np.argsort(ranking_sd)
top_five_indices = sorted_utilities[-1:]
pd.DataFrame(desc[top_five_indices, :]).rename(columns={0: 'Number', 1: 'Name'})

,Number,Name
0,15,awabi (abalone)


## Part 2: Common-sense solutions

### a) Suppose that we simply offer all sushi products, that is, we set S = {1, 2, . . . , n}. What is the expected revenue in this case? 

In [361]:
nopurchase_utilities = np.asarray([3]*nCustomers)
utilities_mat = np.concatenate( (utilities, nopurchase_utilities[:,None]), axis = 1)

In [356]:
def expected_revenue(S):
    # Add the no-purchase option:
    # NB: the products are numbered from 0 to 99. Index 100 ( = nProducts) will correspond to the no-purchase option. 
    S2 = np.append(S, nProducts)
    
    choice_probabilities = {}
    for i in S2:
        choice_probabilities[i] = 0.0;
    
    for k in range(nCustomers):
        ind = np.argmax( utilities_mat[k,S2])
        choice_probabilities[ S2[ind] ] += 1.0/nCustomers
        
    exp_revenue = sum( [choice_probabilities[i] * info[i][2] for i in S])
        
    return exp_revenue, choice_probabilities


S_check = [0,1,2,3,4]
exp_revenue_check, choice_probabilities_check = expected_revenue( S_check )
print("Expected revenue if we offer the set of sushis S = {1, 2, 3, 4, 5}: ", exp_revenue)

Expected revenue if we offer the set of sushis S = {1, 2, 3, 4, 5}:  21.509720000000016


In [360]:
S_all = np.array([i for i in range(nProducts)])

exp_revenue_all, choice_probabilities_all = expected_revenue( S_all )
print("Expected revenue if we simply offer all sushi products: ", exp_revenue_all)

Expected revenue if we simply offer all sushi products:  21.509720000000016


### b) Suppose that we offer the ten highest revenue sushis, that is, we set S = {$i_1$, . . . , $i_{10}$}, where $r_{i_1}$ ≥ $r_{i_2}$ ≥ · · · ≥ $r_{i_n}$ . Which are the ten highest revenue sushis? What is the expected revenue in this case? 

In [350]:
sorted_revenue_increasing = np.argsort(info[:,2])

sorted_revenue_decreasing = np.flip(sorted_revenue_increasing, axis=0)

sorted_revenue_top_10 = sorted_revenue_decreasing[:10]

desc_sorted_revenue_top_10 = desc[sorted_revenue_top_10]

pd.DataFrame(desc_sorted_revenue_top_10).rename(columns={0: 'Number', 1: 'Name'})

,Number,Name
0,9,toro (fatty tuna)
1,15,awabi (abalone)
2,48,tarabagani (king crab)
3,13,akagai (ark shell)
4,5,uni (sea urchin)
5,80,kurumaebi (prawn)
6,20,chu_toro (mildly_fatty tuna)
7,47,suzuki (sea bass)
8,21,hirame (flatfish)
9,42,botanebi (BOTAN shrimp)


In [362]:
S_top_10 = sorted_revenue_top_10.tolist()

exp_revenue_top_10, choice_probabilities_top_10 = expected_revenue( S_top_10 )
print("Expected revenue if we offer the ten highest revenue sushis: ", exp_revenue_top_10)

Expected revenue if we offer the ten highest revenue sushis:  25.63944000000002


### c) Suppose that for every customer k, we determine his/her most preferred sushi, $i^*_k$ = arg $max_{1≤i≤n}  u_{k,i}$. We then offer all of the most preferred sushis, that is, we set S = {$i^*_1$, $i^*_2$, . . . , $i^*_K$}. (Note that some sushis may be the top choice of more than one customer.) What is the expected revenue in this case? What do you notice about this value? 

In [363]:
S_every_perferred = np.unique(np.argmax(utilities[:,0:nProducts], axis = 1)).tolist()

exp_revenue_every_perferred, choice_probabilities_every_perferred = expected_revenue( S_every_perferred )
print("Expected revenue if for every customer k, we determine his/her most preferred sushi:", exp_revenue_every_perferred)

Expected revenue if for every customer k, we determine his/her most preferred sushi: 21.509720000000016


This value is exactly the same as simply offering all sushi products because customers tend to select their most preferred items although offered everything.

### d) Explain why the solution in (a) may be suboptimal. 

By restricting customers' access to their preferred menu choices, we incentivized them to opt for higher-priced sushi items, as offering all choices may lead customers to prioritize personal preferences over revenue-maximizing selections.


### e) Explain why the solution in (b) may be suboptimal.


When offered only the top 10 highest revenue sushis, some customers might find the utility of not purchasing to be higher than selecting from those options, particularly if they are price-sensitive, suggesting potential revenue improvement through the inclusion of additional attractive sushi options for such customers.



## Part 3: An integer optimization model

### a) Explain how constraints (14b) – (14e) correctly model the preferences of each customer.


- 14b: Each customer can choose exactly one item

- 14c: The utility of each customer choosing any item(s) must be at least the utility of the customer choosing an item if that item is offered, or at least the utility of the customer not purchasing anything if that item is not offered

- 14d: The utility of each customer choosing any item(s) must be at least the utilities of the customer not purchasing anything

- 14e: Each customer can choose an item only when the item is offered




Constraint (1c) requires that the utility of the sushi that customer type $k$ chooses is at least the utility of sushi $i$ _if_ sushi $i$ is in the assortment, and otherwise is greater than the utility of the no-purchase option ($u_{k,0}$). 

Constraint (1d) requires that the utility of the sushi that customer type $k$ chooses is at least the utility of the no-purchase option.

Constraint (1e) requires that customer type $k$ can choose sushi $i$ only if sushi $i$ is offered in the assortment.


### b) What is the optimal objective value of the relaxation?

In [353]:
from gurobipy import *

m = Model()

# Create the decision variables
x = m.addVars(nProducts, vtype = GRB.BINARY )
y = m.addVars(nCustomers, nProducts+1, vtype = GRB.BINARY)

# Create the constraints:
print("Creating constraints...")
for k in range(nCustomers):
    m.addConstr( sum(y[k,i] for i in range(nProducts+1)) == 1)
    for i in range(nProducts):
        m.addConstr( y[k,i] <= x[i] )
        m.addConstr( quicksum( utilities_mat[k,j] * y[k,j] for j in range(nProducts+1)) >= utilities_mat[k,i] * x[i] + utilities_mat[k,nProducts]*(1 - x[i]))
    m.addConstr( quicksum( utilities_mat[k,j] * y[k,j] for j in range(nProducts+1)) >= utilities_mat[k,nProducts] )



# Create the objective:
print("Creating objective...")
m.setObjective( quicksum(info[i][2] * 1.0/nCustomers * y[k,i] for k in range(nCustomers) for i in range(nProducts)), GRB.MAXIMIZE)

# Update and solve
m.update()

print("Update completed")
m.optimize()


S_optimal = [i for i in range(nProducts) if x[i].x > 0.5]
print("Optimal set of products: ", S_optimal)

optimal_revenue = m.objval
print("Optimal revenue: ",optimal_revenue)

# Verify that optimal profit is correct:
exp_revenue_S_optimal, choice_probabilities_S_optimal = expected_revenue( S_optimal )
print("Optimal revenue (via expected_revenue): ", exp_revenue_S_optimal)

Creating constraints...
Creating objective...
Update completed
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 101000 rows, 50600 columns and 5301000 nonzeros
Model fingerprint: 0x613f8fbb
Variable types: 0 continuous, 50600 integer (50600 binary)
Coefficient statistics:
  Matrix range     [6e-06, 5e+00]
  Objective range  [2e-02, 7e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 2 rows and 35694 columns (presolve time = 5s) ...
Presolve removed 73359 rows and 36639 columns (presolve time = 10s) ...
Presolve removed 73429 rows and 36655 columns
Presolve time: 12.30s
Presolved: 27571 rows, 13945 columns, 403592 nonzeros
Variable types: 0 continuous, 13945 integer (13945 binary)
Found heuristic solution: objective 21.5232800
Found heuristic solution: objective

In [346]:
m_relaxed = m.relax()

m_relaxed.optimize()

relaxation_bound = m_relaxed.objval

print("Relaxation bound: ", relaxation_bound)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 101000 rows, 50600 columns and 5301000 nonzeros
Model fingerprint: 0x56828d7b
Coefficient statistics:
  Matrix range     [6e-06, 5e+00]
  Objective range  [2e-02, 7e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve time: 3.64s
Presolved: 101000 rows, 50600 columns, 5301000 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...


Barrier performed 0 iterations in 4.77 seconds (4.60 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
    1156    3.0587512e+01   0.000000e+00   0.000000e+00      5s

Solved in 1156 iterations and 4.83 seconds (5.09 work units)
Optimal objective  3.0

### c) A manager claims that it should be possible to obtain an assortment with an expected percustomer revenue of $\$32$. Explain why this is impossible based on your answer to (b).


This is impossible because the relaxation bound is $\$30.58$, which tells us that no product line can obtain a per-customer expected revenue greater than $\$30.58$.


### d) Now, implement the integer version of the above problem (i.e., the binary constraints (14f) and (14g) are enforced) in Python. Solve the problem. What is the expected per-customer revenue of the optimal assortment? How much does this improve over the assortments from Part 2?

In [354]:
optimal_revenue = m.objval
print("Optimal revenue: ",optimal_revenue)

# Verify that optimal profit is correct:
exp_revenue_S_optimal, choice_probabilities_S_optimal = expected_revenue( S_optimal )
print("Optimal revenue (via expected_revenue): ", exp_revenue_S_optimal)

Optimal revenue:  26.24454000000005
Optimal revenue (via expected_revenue):  26.244540000000022


In [365]:
improvement_all = (exp_revenue_S_optimal - exp_revenue_all)/exp_revenue_all * 100
print("This improves over the assortments that offers all sushis from Part 2 by ", improvement_all, "%.")

improvement_top_10 = (exp_revenue_S_optimal - exp_revenue_top_10)/exp_revenue_top_10 * 100
print("This improves over the assortments that offers top 10 sushis from Part 2 by ", improvement_top_10, "%.")

improvement_every_perferred = (exp_revenue_S_optimal - exp_revenue_every_perferred)/exp_revenue_every_perferred * 100
print("This improves over the assortments that offers each customer's preferred sushi from Part 2 by ", improvement_every_perferred, "%.")


This improves over the assortments that offers all sushis from Part 2 by  22.01246692193112 %.
This improves over the assortments that offers top 10 sushis from Part 2 by  2.360035944622814 %.
This improves over the assortments that offers each customer's preferred sushi from Part 2 by  22.01246692193112 %.


### e) What is the optimal set of sushis the restaurant should offer?

In [367]:
pd.DataFrame(desc[S_optimal]).rename(columns={0: 'Number', 1: 'Name'})

,Number,Name
0,9,toro (fatty tuna)
1,15,awabi (abalone)
2,19,saba (mackerel)
3,31,geso (squid feet)
4,48,tarabagani (king crab)
5,73,mentaiko_maki (chili cod roe roll)
6,76,ika_nattou (squid & fermented bean)
7,88,tobiuo (flying fish)
